In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [6]:
#@title Default title text
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git config --global user.email "007rohitjj04@gmail.com"
!git config --global user.name "Rohith04MVK"
!git add -A
!git commit -m "Still testing google colab"
!git push

/content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
[main cb9cbc0] Still testing google colab
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 839 bytes | 419.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/Rohith04MVK/Object-Detection-Website.git
   84dad0b..cb9cbc0  main -> main
